# TT Clustering

Clustering over the TT datasets. These are categorical labels if two points in the time series interact as they should. If two points in cause and effect goes in the same direction, these are labeled as 1, if there is no relation, for example the cause has 1.5 and effect 0 implies no effect between them, so they are labeled as 0 and if its behaviour is opposite, these are labeled as -1

## 1. Preprocessing

In [8]:
import gc
import pandas as pd
import glob
import sklearn
import numpy as np

In [10]:
files=glob.glob("../data/Tt/*")
posLabels = glob.glob("../LabelOutputs/*")

In [25]:
trainIndexFiles=glob.glob("../Train/*")
frames = list()
for f in trainIndexFiles:
    indices = pd.read_csv(f,sep=";")
    indexList=list(indices['0'])
    protein = f.split("/")[-1].split(".")[0]
    fpath = '../data/Tt/'+f.split("/")[-1].split(".")[0]+".csv"
    TtRead = pd.read_csv(fpath,sep=";")
    TtPartialTrain=TtRead[TtRead.columns[5:]].loc[indexList].copy()
    TtPartialTrain.insert(0,column='Protein',value =protein )
    del TtRead
    gc.collect()
    frames.append(TtPartialTrain)

In [29]:
ttTrain = pd.concat(frames)

In [30]:
ttTrain.head()

,Protein,C0_C15~E0_E15,C15_C30~E15_E30,C30_C90~E30_E90,C90_C120~E90_E120,C120_C180~E120_E180,C180_C210~E180_E210,C210_C240~E210_E240,C240_C360~E240_E360,C0_C15~E15_E30,...,C0_C240~E15_E360,C0_C240~E0_E360,C0_C360~E240_E360,C0_C360~E210_E360,C0_C360~E180_E360,C0_C360~E120_E360,C0_C360~E90_E360,C0_C360~E30_E360,C0_C360~E15_E360,C0_C360~E0_E360
1530,LEF1,1,1,0,-1,0,0,-1,-1,1,...,-1,-1,1,0,1,1,0,-1,-1,-1
1531,LEF1,0,0,1,0,-1,1,0,1,0,...,-1,0,1,1,1,1,-1,-1,-1,0
1532,LEF1,1,1,-1,1,1,1,0,0,-1,...,-1,-1,0,0,1,0,1,-1,-1,-1
1533,LEF1,0,0,0,0,-1,0,1,-1,0,...,1,1,0,0,0,0,0,0,0,0
1535,LEF1,1,1,0,1,-1,1,0,-1,0,...,-1,-1,-1,-1,-1,0,0,1,1,1


## 2. Clustering

We will use simple clustering with Hamming distance

In [36]:
!pip install hdbscan

    100% |████████████████████████████████| 4.7MB 123kB/s ta 0:00:01
  Running setup.py bdist_wheel for hdbscan ... done
  Stored in directory: /home/a20125359/.cache/pip/wheels/3c/66/6b/d47e0a5400f27f6cb22977e0e3c8826bc7e716c5b3f3ade0cb
Successfully built hdbscan


In [37]:
import hdbscan

In [41]:
clusterer = hdbscan.HDBSCAN(algorithm='best', alpha=1.0, approx_min_span_tree=True,
    gen_min_span_tree=False, leaf_size=40,
    metric='hamming', min_cluster_size=5, min_samples=None, p=None)

In [ ]:
clusterer.fit(ttTrain[ttTrain.columns[1:]])